In [1]:
import json
import random
import torch

In [3]:
import pandas as pd

dataset = pd.read_csv("data_full_v1.csv")

In [4]:
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)

dataset['prompt'] = dataset.apply(lambda row: (row['prompt']).removeprefix('Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: '), axis=1)


In [5]:
dataset['prompt'] = dataset['prompt'].str.lower()

In [6]:
dataset

,id,prompt,label,ground_truth
0,4,husband has a direct effect on wife and alarm ...,yes,1
1,7,husband has a direct effect on wife and alarm ...,no,0
2,8,husband has a direct effect on wife and alarm ...,yes,1
3,15,husband has a direct effect on wife and alarm ...,no,0
4,21,husband has a direct effect on wife and alarm ...,no,0
...,...,...,...,...
10107,31012,zuph has a direct effect on glimx. jyka has a ...,yes,1
10108,31014,zuph has a direct effect on glimx. jyka has a ...,yes,1
10109,31015,zuph has a direct effect on glimx. jyka has a ...,no,0
10110,31016,zuph has a direct effect on glimx. jyka has a ...,no,0


In [7]:
dataset = json.loads(dataset.to_json(orient ='records'))

# Shuffle the dataset
random.shuffle(dataset)

# Define split sizes for a 75-25 split
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size

# Split the dataset
train_data = dataset[:train_size]
validation_data = dataset[train_size:]

In [8]:
from torch.utils.data import Dataset

class CladderDataset(Dataset):
    def __init__(self, cladder_data, tokenizer):
        self.cladder_data = cladder_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.cladder_data)

    def __getitem__(self, idx):
        data = self.cladder_data[idx]
        # print(data)
        prompt = data['prompt']
        label = data['ground_truth']

        # print(prompt)
        # print(label)

        # Tokenize
        encoded_input = self.tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').to("cuda")
        # print(encoded_input)
        encoded_input['labels'] = torch.tensor(label).to("cuda")

        return encoded_input


In [9]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch
# from torch.utils.data import DataLoader
# import torch.optim as optim

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
# model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels = 2)

In [10]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

In [11]:
# from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from transformers import AutoTokenizer as tkz
from transformers import AutoModelForSequenceClassification as mpc

# Load tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

# run = 0

for run in range(3):
  model_base = 'distilroberta-base'
  tokenizer = tkz.from_pretrained(model_base)
  model = mpc.from_pretrained(model_base, num_labels = 2)

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  # Prepare the DataLoader
  cladder_dataset = CladderDataset(train_data, tokenizer)
  dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

  # Optimizer and Loss Function
  optimizer = optim.Adam(model.parameters(), lr=1e-5)
  loss_fn = torch.nn.CrossEntropyLoss()



  # Training Loop
  model.train()

  #change to 3

  for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  torch.save(model.state_dict(), model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Epoch: 0, Loss: 0.683819055557251
Epoch: 0, Loss: 0.6654972434043884
Epoch: 0, Loss: 0.7067925333976746
Epoch: 0, Loss: 0.6728636622428894
Epoch: 0, Loss: 0.7357813715934753
Epoch: 0, Loss: 0.6971317529678345
Epoch: 0, Loss: 0.6472177505493164
Epoch: 0, Loss: 0.7359126806259155
Epoch: 0, Loss: 0.7766394019126892
Epoch: 0, Loss: 0.6938063502311707
Epoch: 0, Loss: 0.6666204929351807
Epoch: 0, Loss: 0.6896722316741943
Epoch: 0, Loss: 0.6666810512542725
Epoch: 0, Loss: 0.7627447843551636
Epoch: 0, Loss: 0.7024754285812378
Epoch: 0, Loss: 0.7171366214752197
Epoch: 0, Loss: 0.7023681402206421
Epoch: 0, Loss: 0.7042354345321655
Epoch: 0, Loss: 0.669947624206543
Epoch: 0, Loss: 0.6791932582855225
Epoch: 0, Loss: 0.6769875884056091
Epoch: 0, Loss: 0.7025024890899658
Epoch: 0, Loss: 0.6732994318008423
Epoch: 0, Loss: 0.6528065800666809
Epoch: 0, Loss: 0.6924160718917847
Epoch: 0, Loss: 0.7254899740219116
Epoch: 0, Loss: 0.6881948113441467
Epoch: 0, Loss: 0.7083768248558044
Epoch: 0, Loss: 0.6747

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.6765614748001099
Epoch: 0, Loss: 0.6955424547195435
Epoch: 0, Loss: 0.6465030312538147
Epoch: 0, Loss: 0.6990805268287659
Epoch: 0, Loss: 0.721595287322998
Epoch: 0, Loss: 0.7275596857070923
Epoch: 0, Loss: 0.7214282155036926
Epoch: 0, Loss: 0.7642388939857483
Epoch: 0, Loss: 0.7065451145172119
Epoch: 0, Loss: 0.7368901371955872
Epoch: 0, Loss: 0.7169536352157593
Epoch: 0, Loss: 0.7424879670143127
Epoch: 0, Loss: 0.6770749092102051
Epoch: 0, Loss: 0.7006558179855347
Epoch: 0, Loss: 0.6783448457717896
Epoch: 0, Loss: 0.6991620063781738
Epoch: 0, Loss: 0.6995931267738342
Epoch: 0, Loss: 0.685480535030365
Epoch: 0, Loss: 0.6929247379302979
Epoch: 0, Loss: 0.7070684432983398
Epoch: 0, Loss: 0.7010186910629272
Epoch: 0, Loss: 0.7132654190063477
Epoch: 0, Loss: 0.7151639461517334
Epoch: 0, Loss: 0.6866148114204407
Epoch: 0, Loss: 0.7089729905128479
Epoch: 0, Loss: 0.6954739093780518
Epoch: 0, Loss: 0.6862086057662964
Epoch: 0, Loss: 0.6939209699630737
Epoch: 0, Loss: 0.6925

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.7414270639419556
Epoch: 0, Loss: 0.7944308519363403
Epoch: 0, Loss: 0.6296147108078003
Epoch: 0, Loss: 0.7307814359664917
Epoch: 0, Loss: 0.7603018283843994
Epoch: 0, Loss: 0.7204824090003967
Epoch: 0, Loss: 0.7261050343513489
Epoch: 0, Loss: 0.6913033127784729
Epoch: 0, Loss: 0.7226462960243225
Epoch: 0, Loss: 0.6814948916435242
Epoch: 0, Loss: 0.7325550317764282
Epoch: 0, Loss: 0.6701757907867432
Epoch: 0, Loss: 0.671437680721283
Epoch: 0, Loss: 0.6833991408348083
Epoch: 0, Loss: 0.70065838098526
Epoch: 0, Loss: 0.714216411113739
Epoch: 0, Loss: 0.6967467665672302
Epoch: 0, Loss: 0.6970133781433105
Epoch: 0, Loss: 0.6783298850059509
Epoch: 0, Loss: 0.6874213814735413
Epoch: 0, Loss: 0.658504843711853
Epoch: 0, Loss: 0.6943249702453613
Epoch: 0, Loss: 0.6984883546829224
Epoch: 0, Loss: 0.69124835729599
Epoch: 0, Loss: 0.7142348289489746
Epoch: 0, Loss: 0.6775708794593811
Epoch: 0, Loss: 0.6906585693359375
Epoch: 0, Loss: 0.7291063666343689
Epoch: 0, Loss: 0.723526835

In [12]:
# # Prepare the DataLoader
# cladder_dataset = CladderDataset(train_data, tokenizer)
# dataloader = DataLoader(cladder_dataset, batch_size=8, shuffle=True)

# # Optimizer and Loss Function
# optimizer = optim.Adam(model.parameters(), lr=1e-5)
# loss_fn = torch.nn.CrossEntropyLoss()

In [13]:
# # Training Loop
# model.train()

# for epoch in range(3):  # Number of epochs
#   for batch in dataloader:
#       optimizer.zero_grad()

#       input_ids = batch['input_ids'].squeeze(1)
#       attention_mask = batch['attention_mask'].squeeze(1)
#       labels = batch['labels']

#       outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#       loss = outputs.loss

#       loss.backward()
#       optimizer.step()

#       print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

overall_accuracies, accuracies, f1s, precisions, recalls = [], [], [], [], []

for run in range(3):
  model_path = 'cladder_' + model_base + '_' + str(run) + '.pt'
  print(model_path)

  model = mpc.from_pretrained(model_base, num_labels = 2)
  model.load_state_dict(torch.load(model_path))

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)

  tokenizer = tkz.from_pretrained(model_base)
  validation_dataset = CladderDataset(validation_data, tokenizer)
  validation_dataloader = DataLoader(validation_dataset, batch_size=8)




  model.eval()
  correct_predictions = 0
  total_predictions = 0

  accuracy, f1, precision, recall = [], [], [], []
  batches = 0

  def compute_metrics(y_true, y_pred):
      accuracy = accuracy_score(y_true, y_pred)
      # print(y_true)
      # print(y_pred)
      f1 = f1_score(y_true, y_pred)

      precision = precision_score(y_true, y_pred)
      # print(precision)
      recall = recall_score(y_true, y_pred)
      return [accuracy, f1, precision, recall]

  with torch.no_grad():
      for batch in validation_dataloader:
          input_ids = batch['input_ids'].squeeze(1)
          attention_mask = batch['attention_mask'].squeeze(1)
          labels = batch['labels'].cpu()  # ground truth
          # print(labels)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          predictions = torch.argmax(outputs.logits, dim=1).cpu()  # predictions
          # print(predictions)

          calculated_metrics = compute_metrics(labels, predictions)
          accuracy.append(calculated_metrics[0])
          f1.append(calculated_metrics[1])
          precision.append(calculated_metrics[2])
          recall.append(calculated_metrics[3])
          batches += 1

          correct_predictions += (predictions == labels).sum().item()
          total_predictions += labels.size(0)

  overall_accuracy = correct_predictions / total_predictions
  print(f"Overall Accuracy: {overall_accuracy:.4f}")
  overall_accuracies.append(overall_accuracy)

  accuracy_var = sum(accuracy)/batches
  print(f"Accuracy: {sum(accuracy)/batches:.4f}")
  print(accuracy_var)
  accuracies.append(accuracy_var)

  f1_var = sum(f1)/batches
  print(f"F1: {sum(f1)/batches:.4f}")
  print(f1_var)
  f1s.append(f1_var)

  precision_var = sum(precision)/batches
  print(f"Precision: {sum(precision)/batches:.4f}")
  print(precision_var)
  precisions.append(precision_var)

  recall_var = sum(recall)/batches
  print(f"Recall: {sum(recall)/batches:.4f}")
  print(recall_var)
  recalls.append(precision_var)

  print(f'Number of batches: {batches}')

cladder_distilroberta-base_0.pt


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Overall Accuracy: 0.5051
Accuracy: 0.5051
0.5051424050632911
F1: 0.6505
0.6505039264532935
Precision: 0.5051
0.5051424050632911
Recall: 0.9968
0.9968354430379747
Number of batches: 316
cladder_distilroberta-base_1.pt


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/met

Overall Accuracy: 0.5447
Accuracy: 0.5447
0.5446993670886076
F1: 0.1695
0.16948965240104472
Precision: 0.3645
0.364451476793249
Recall: 0.1166
0.11656871609403256
Number of batches: 316
cladder_distilroberta-base_2.pt


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/met

Overall Accuracy: 0.4988
Accuracy: 0.4988
0.4988132911392405
F1: 0.0669
0.06694293751255775
Precision: 0.1456
0.14556962025316456
Recall: 0.0461
0.0461045810729355
Number of batches: 316


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

In [ ]:
validation_dataset = CladderDataset(validation_data, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=8)

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        print(f"predictions: {predictions}, labels: {labels}")
        total_predictions += labels.size(0)

print(f"total_predictions: {total_predictions}")

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")


predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), labels: tensor([1, 0, 0, 1, 1, 0, 0, 0], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), labels: tensor([1, 1, 0, 1, 1, 1, 1, 1], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), labels: tensor([1, 0, 0, 0, 1, 0, 0, 0], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), labels: tensor([0, 0, 0, 0, 1, 0, 0, 1], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), labels: tensor([1, 1, 1, 1, 1, 0, 1, 1], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), labels: tensor([0, 0, 1, 1, 1, 0, 1, 1], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), labels: tensor([1, 0, 1, 0, 1, 1, 0, 1], device='cuda:0')
predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), labels: tensor([1, 1, 1, 0, 0, 1, 0, 1], device='cuda:0')
predictions: tensor([0, 